サーバレスコンピュートで実行してください

In [0]:
%run ./00_config

## 1. Genie用テーブル作成（silver相当）

In [0]:
import os
import shutil
from pyspark.sql import functions as F

# カレントディレクトリ取得
current_dir_path = os.getcwd()
src_no_scheme = f"{current_dir_path}/Sample-Superstore_Japanese.csv"

# csvをボリュームにコピー
dest_path = f"/Volumes/{catalog}/{schema}/{volume}/Sample-Superstore_Japanese.csv"
shutil.copyfile(src_no_scheme, dest_path)

# CSV を Spark Dataframe に読み込み
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(dest_path)
)

# 日本語列名を英語にリネーム
mapping = {
    "行 ID": "row_id",
    "オーダー ID": "order_id",
    "オーダー日": "order_date",
    "出荷日": "ship_date",
    "出荷モード": "ship_mode",
    "顧客 ID": "customer_id",
    "顧客名": "customer_name",
    "顧客区分": "customer_segment",
    "市区町村": "city",
    "都道府県": "prefecture",
    "国/領域": "country_region",
    "地域": "region",
    "製品 ID": "product_id",
    "カテゴリ": "category",
    "サブカテゴリ": "sub_category",
    "製品名": "product_name",
    "売上": "sales_amount",
    "数量": "quantity",
    "割引率": "discount_rate",
    "利益": "profit",
    "緯度": "latitude",
    "経度": "longitude",
    "地域マネージャー": "region_manager",
    "予算": "budget",
    "返品": "returned",
}

for jp, en in mapping.items():
    if jp in df.columns:
        df = df.withColumnRenamed(jp, en)

# 利益がマイナスの行を除外
df = df.filter(F.col("profit") >= 0)

# 返品フラグ列（returnedg: ○/NULL → returned: true/false）
df = df.withColumn(
    "returned",  # 返品フラグ（真偽値）
    F.when(F.col("returned") == "○", F.lit(True)).otherwise(F.lit(False))
)

# カラム並び順を整える
df_silver = df.select(
    "order_id",         # オーダー ID
    "row_id",           # 行 ID
    "order_date",       # オーダー日
    "ship_date",        # 出荷日
    "ship_mode",        # 出荷モード
    "customer_id",      # 顧客 ID
    "customer_name",    # 顧客名
    "customer_segment", # 顧客区分
    "city",             # 市区町村
    "prefecture",       # 都道府県
    "country_region",   # 国/地域
    "region",           # 地域
    "region_manager",   # 地域マネージャー
    "product_id",       # 製品 ID
    "category",         # カテゴリ
    "sub_category",     # サブカテゴリ
    "product_name",     # 製品名
    "sales_amount",     # 売上
    "quantity",         # 数量
    "discount_rate",    # 割引率
    "profit",           # 利益
    "latitude",         # 緯度
    "longitude",        # 経度
    "budget",           # 予算
    "returned",         # 返品フラグ（true/false）
)

# Silver テーブルとして保存
df_silver.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalog}.{schema}.sales_silver")

print(df_silver.count())
df_silver.display()
df_silver.printSchema()

In [0]:
# 変数定義
TABLE_PATH = f'{catalog}.{schema}.sales_silver'      # テーブルパス
PK_CONSTRAINT_NAME = 'pk_sales_silver'               # 主キー名

# NOT NULL 制約の追加（order_id を主キーにする想定）
columns_to_set_not_null = [
    'order_id'
]

for column in columns_to_set_not_null:
    spark.sql(f"""
    ALTER TABLE {TABLE_PATH}
    ALTER COLUMN {column} SET NOT NULL;
    """)

# 主キー設定（既存の制約があれば削除してから追加）
spark.sql(f'''
ALTER TABLE {TABLE_PATH} DROP CONSTRAINT IF EXISTS {PK_CONSTRAINT_NAME};
''')

spark.sql(f'''
ALTER TABLE {TABLE_PATH}
ADD CONSTRAINT {PK_CONSTRAINT_NAME} PRIMARY KEY (order_id);
''')

# チェック（必要ならコメントアウト外して確認）
# display(
#     spark.sql(f'''
#     DESCRIBE EXTENDED {TABLE_PATH}
#     '''))

In [0]:
# テーブル名
table_name = f"{catalog}.{schema}.sales_silver"

# テーブルコメント（ダブルクォート内のダブルクォートをエスケープ）
comment = """
テーブル名：sales_silver / 売上サマリ（注文×返品×地域マネージャー）
説明：スーパーストアの注文データに対して、返品情報および地域マネージャー情報を付与したダッシュボード向けの Silver テーブルです。顧客・製品・地域・期間別の売上・利益・返品状況を分析するための基礎データを提供します。
補足：
* 主キー：order_id
* returns テーブル、region_managers テーブルと統合済み
"""
spark.sql(f'COMMENT ON TABLE {table_name} IS "{comment}"')

# カラムコメント
column_comments = {
    "order_id":         "オーダー ID（主キー）",
    "row_id":           "行 ID",
    "order_date":       "オーダー日（YYYY-MM-DD）",
    "ship_date":        "出荷日（YYYY-MM-DD）",
    "ship_mode":        "出荷モード",
    "customer_id":      "顧客 ID",
    "customer_name":    "顧客名",
    "customer_segment": "顧客区分",
    "city":             "市区町村",
    "prefecture":       "都道府県",
    "country_region":   "国/領域",
    "region":           "地域",
    "region_manager":   "地域マネージャー",
    "product_id":       "製品 ID",
    "category":         "カテゴリ",
    "sub_category":     "サブカテゴリ",
    "product_name":     "製品名",
    "sales_amount":     "売上金額（数値）",
    "quantity":         "数量（数値）",
    "discount_rate":    "割引率（0〜1 の実数）",
    "profit":           "利益（数値）",
    "latitude":         "緯度（数値：度）",
    "longitude":        "経度（数値：度）",
    "budget":           "予算金額（数値）",
    "returned":         "返品フラグ（真偽値）",
}

for column, col_comment in column_comments.items():
    escaped_comment = col_comment.replace("'", "\\'")
    sql_query = f"ALTER TABLE {table_name} ALTER COLUMN {column} COMMENT '{escaped_comment}'"
    spark.sql(sql_query)

In [0]:
# spark.sql(f"""
# SELECT
#   order_id,
#   -- product_id,
#   product_name,
#   sales_amount,
#   quantity,
#   discount_rate,
#   profit
# FROM {catalog}.{schema}.sales_silver
# WHERE profit < 0
# ORDER BY profit;
# """).display()

### 2. SQL関数をUCに登録する

関数：`セグメント×出荷モードの総合スコアリング関数`<br>
関数型：`segment_shipmode_performance(start_date DATE, end_date DATE)`<br>
やること：<br>
期間内のデータを対象に顧客セグメント × 出荷モードごとに
- `売上・利益・利益率`
- `平均リードタイム（日数）`
- `返品率`
- `予算達成度（売上 ÷ budget）` を集計
- `総合パフォーマンス評価タグ`をつける（例：'⭐ High-performing' / '⚠ Needs attention' / 'Average'）

Genieでのプロンプト例：<br>
`2018年〜2019年の、High-performing と Needs attention の組み合わせを抜き出して、それぞれビジネス観点でコメントしてください。`

In [0]:
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog}.{schema}.segment_shipmode_performance(
  -- Genie が扱えるよう、引数は STRING 型（YYYY-MM-DD形式想定）
  start_date STRING,
  end_date   STRING
)
RETURNS TABLE (
  customer_segment     STRING,
  ship_mode            STRING,
  order_cnt            BIGINT,
  total_sales_amount   DOUBLE,
  total_profit         DOUBLE,
  profit_margin        DOUBLE,
  avg_lead_time_days   DOUBLE,
  return_rate          DOUBLE,
  total_budget         DOUBLE,
  budget_achievement   DOUBLE,
  performance_tag      STRING
)
COMMENT '顧客セグメント×出荷モード別に主要KPIを集計し、指定期間または直近2年間の総合パフォーマンス評価（優秀／標準／要改善）を返す関数。'
RETURN
WITH params AS (
  SELECT
    -- ★ start_date が NULL or '' の場合 → 直近2年間の開始日
    COALESCE(
      NULLIF(start_date, ''), 
      CAST(date_sub(current_date(), 365*2) AS STRING)
    ) AS start_date_str,

    -- ★ end_date が NULL or '' の場合 → 今日
    COALESCE(
      NULLIF(end_date, ''), 
      CAST(current_date() AS STRING)
    ) AS end_date_str
),
base AS (
  SELECT
    customer_segment,
    ship_mode,
    DATEDIFF(TO_DATE(ship_date), TO_DATE(order_date)) AS lead_time_days,
    sales_amount,
    profit,
    budget,
    CASE WHEN returned THEN 1 ELSE 0 END AS returned_flag
  FROM {catalog}.{schema}.sales_silver, params
  WHERE TO_DATE(order_date)
        BETWEEN TO_DATE(params.start_date_str) AND TO_DATE(params.end_date_str)
),
agg AS (
  SELECT
    customer_segment,
    ship_mode,
    COUNT(*)            AS order_cnt,
    SUM(sales_amount)   AS total_sales_amount,
    SUM(profit)         AS total_profit,
    SUM(budget)         AS total_budget,
    AVG(lead_time_days) AS avg_lead_time_days,
    AVG(returned_flag)  AS return_rate
  FROM base
  GROUP BY customer_segment, ship_mode
),
scored AS (
  SELECT
    customer_segment,
    ship_mode,
    order_cnt,
    total_sales_amount,
    total_profit,
    CASE 
      WHEN total_sales_amount = 0 THEN NULL
      ELSE total_profit / total_sales_amount
    END AS profit_margin,
    avg_lead_time_days,
    return_rate,
    total_budget,
    CASE 
      WHEN total_budget = 0 THEN NULL
      ELSE total_sales_amount / total_budget
    END AS budget_achievement,
    CASE 
      WHEN (total_profit / NULLIF(total_sales_amount, 0)) >= 0.2
           AND return_rate < 0.05
           AND avg_lead_time_days <= 3
        THEN '優秀'
      WHEN (total_profit / NULLIF(total_sales_amount, 0)) < 0.05
           OR return_rate > 0.1
        THEN '要改善'
      ELSE '標準'
    END AS performance_tag
  FROM agg
)
SELECT * FROM scored;
""")

In [0]:
df = spark.sql(f"""
SELECT *
FROM {catalog}.{schema}.segment_shipmode_performance('2016-01-01','2020-12-31')
LIMIT 10;
""")

display(df)